In [1]:
import requests
import pandas
from pandas import to_numeric
from bs4 import BeautifulSoup
from pykrx import stock as krx
import FinanceDataReader as fdr
from datetime import datetime, date, timedelta

import json
import urllib.request
from collections import namedtuple
from tqdm import tqdm
import sqlite3
from sqlite3 import Connection, Cursor, IntegrityError
from typing import NamedTuple

In [111]:
DBFILE = 'd:/tmp/stock_prices.db'

In [2]:
# https://joo-ramzzi.tistory.com/7
# https://dkgkim.tistory.com/21
# https://github.com/sharebook-kr/pykrx

## Utility functions

In [3]:
class StockInfo(NamedTuple):
    종목명: str
    현재가: int
    시총: str
    market_cap: int
    PER: int
    PBR: int

In [4]:
def get_price(code: str, cnt: int, period :str='day'):
    response = requests.get(f"https://fchart.stock.naver.com/sise.nhn?symbol={code}&requestType=0&count={cnt}&timeframe={period}")
    bs = BeautifulSoup(response.text, "html.parser")
    items = bs.find_all('item')
    price_data = []
    for item in items:
        data = item['data'].split('|')
        price_data.append(data)

    price_data = pandas.DataFrame(price_data, columns=["날짜", "시가", "고가", "저가", "종가", "거래량"])
    price_data.index = price_data["날짜"]
    price_data = price_data[["시가", "고가", "저가", "종가", "거래량"]]\
                        .rename(columns={
                            '시가':'Open',
                            '고가':'High',
                            '저가':'Low',
                            '종가':'Close',
                            '거래량':'Volume'
                        })
    price_data = price_data.apply(to_numeric)
    return price_data.iloc[::-1]

In [5]:
def get_stock_codes():
    code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', encoding='cp949')[0]
    code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
    return code_df

In [6]:
def market_capitalization(item_code: str):
    url = "https://m.stock.naver.com/api/stock/%s/integration"%(item_code)
    #urllib.request를 통해 링크의 결과를 가져옵니다.
    raw_data = urllib.request.urlopen(url).read()
    #추후, 데이터 가공을 위해 json 형식으로 변경 합니다.
    json_data = json.loads(raw_data)

    #종목명 가져오기
    stock_name = json_data['stockName']

    #가격 가져오기
    current_price = json_data['dealTrendInfos'][0]['closePrice']

    #시총 가져오기
    for code in json_data['totalInfos']:
        if 'marketValue' == code['code']:
            marketSum_value = code['value']

    #PER 가져오기
    for i in json_data['totalInfos']:
        if 'per' == i['code']:
            per_value_str = i['value']

    #PBR 가져오기
    for v in json_data['totalInfos']:
        if 'pbr' == v['code']:
            pbr_value_str = v['value']

    return StockInfo(stock_name, current_price, marketSum_value, int(''.join([d for d in list(marketSum_value) if d in '0123456789'])), per_value_str, pbr_value_str)

In [7]:
def resample_df(df, period='W-FRI'):
    wm_df = pd.DataFrame()
    wm_df = pd.concat([wm_df, df[['Open']].resample(period).first()], axis=1)
    wm_df = pd.concat([wm_df, df[['High']].resample(period).max()], axis=1)
    wm_df = pd.concat([wm_df, df[['Low']].resample(period).min()], axis=1)
    wm_df = pd.concat([wm_df, df[['Close']].resample(period).last()], axis=1)
    wm_df = pd.concat([wm_df, df[['Volume']].resample(period).mean()], axis=1)
    return wm_df

In [8]:
# df_codes = get_stock_codes()
# df_ks_index = fdr.DataReader('KS11', date.today()-timedelta(days=480))[::-1]
# df_kq_index = fdr.DataReader('KQ11', date.today()-timedelta(days=480))[::-1]
# df_ks_stocks = fdr.StockListing('KOSPI') # KOSPI: 940 종목
# df_kq_stocks = fdr.StockListing('KOSDAQ') # KOSDAQ: 1,597 종목

## 사용법

```python
get_price("035720", 10, "week")

get_price(df_codes.종목코드[23], 10, 'week')

market_capitalization("373220")

stock.get_market_ohlcv("20200831", market="KOSPI") # KOSPI의 개별 종목 별 시고저종

# KOSPI Index 코스피 지수 데이터 
kospi_df = fdr.DataReader('KS11')
week_kospi_df = get_week_month_df(df, period='W-MON')
month_kospi_df = get_week_month_df(df, period='M')
```
출처: https://simpling.tistory.com/69 [simpling:티스토리]

In [9]:
def kospi_strong_stocks():    
    df_ks_index_week = resample_df(df_ks_index, period='W-FRI')[::-1]
    date = df_ks_index_week[:52]["Low"].idxmin()
    lowest = df_ks_index_week.loc[date, "Low"]
    currval = df_ks_index.iloc[0]["Close"] 
    ks_rate = (currval - lowest)/lowest*100
    res = []
    for code in tqdm(df_ks_stocks["Code"]):
        df_week = get_price(code, 52, "week")
        date = df_week[:52]["Low"].idxmin()
        lowest = df_week.loc[date, "Low"]
        currval = get_price(code, 1, "day").iloc[0]["Close"]
        rate = (currval - lowest)/lowest*100
        if (rate > ks_rate):            
            res.append([code, rate, market_capitalization(code).market_cap])
    
    return ks_rate, pd.DataFrame(res, columns=["code", "rate", "marcap"])

In [10]:
from pydantic import BaseModel

class StockPrice(BaseModel):
    code: str
    market: str
    date: str
    open: float
    high: float
    low: float
    close: float
    volume: float

In [11]:
def db_insert(curs: Cursor, info: StockPrice):
    qry = "insert into StockPrice values" \
          "(:code, :market, :date, :open, :high, :low, :close, :volume)"
    params = info.model_dump()
    curs.execute(qry, params)

In [143]:
def get_ohlc(curs: Cursor, code: str, start_date: str|None=None, end_date: str|None=None):
    qry = "select * from StockPrice where code=:code "
    if start_date and end_date:
        qry += f"AND date BETWEEN '{start_date}' and '{end_date}'"
    params = {"code": code}
    curs.execute(qry, params)
    res = curs.fetchall()
    df = pd.DataFrame(res, columns=["Code", "Market", "Date", "Open", "High", "Low", "Close", "Volume"])
    return df

In [134]:
def get_stock_list(curs: Cursor, market: str):
    qry = f"select DISTINCT code from StockPrice where market='{market}'"
    curs.execute(qry)
    return curs.fetchall() 

In [13]:
def build_stock_price_db(dbfile: str, start_date: str, end_date: str):
    conn = sqlite3.connect(dbfile)
    curs = conn.cursor()
    curs.execute("create table if not exists \
    StockPrice(code text, market text, \
    date text, open, high, low, close, volume, constraint pkey primary key(code, date))")
    def build(stock_list: list, market: str):
        for code in tqdm(stock_list, desc=market):
            df = fdr.DataReader(code, start_date, end_date).reset_index()
            for _, row in df.iterrows():
                sp = StockPrice(code=code, 
                                market=market, 
                                date=row['Date'].strftime('%Y-%m-%d'), 
                                open=row['Open'],
                                high=row['High'],
                                low=row['Low'],
                                close=row['Close'],
                                volume=row['Volume'])
                try:
                    db_insert(curs, sp)                    
                except IntegrityError:
                    pass
    build(fdr.StockListing('KOSPI')['Code'], 'kospi')
    build(['KS11'], 'KS11')
    build(fdr.StockListing('KOSDAQ')['Code'], 'kosdaq')
    build(['KQ11'], 'KQ11')
    conn.commit()
    conn.close()

In [14]:
def update_stock_price_db(dbfile: str):
    conn = sqlite3.connect(dbfile)
    curs = conn.cursor()
    qry = "select DISTINCT code, market from StockPrice"
    curs.execute(qry)
    res = curs.fetchall() 
    for code, market in tqdm(res):
        qry = "select MAX(date) from StockPrice where code=:code"
        params = {"code": code}
        curs.execute(qry, params)
        res = curs.fetchall()
        start_date = res[0][0]
        today = date.today()
        if today == start_date:
            continue
        df = fdr.DataReader(code, start_date, today.strftime("%Y-%m-%d")).reset_index()
        for _, row in df.iterrows():
            sp = StockPrice(code=code, 
                            market=market, 
                            date=row['Date'].strftime('%Y-%m-%d'), 
                            open=row['Open'],
                            high=row['High'],
                            low=row['Low'],
                            close=row['Close'],
                            volume=row['Volume'])
            try:
                db_insert(curs, sp)                    
            except IntegrityError:
                pass
    conn.commit()
    conn.close()

In [151]:
def oneday_update_stock_price_db(dbfile: str, date_: str|None = None):
    conn = sqlite3.connect(dbfile)
    curs = conn.cursor()
    if not date_:
        date_ = date.today().strftime("%Y-%m-%d")
        
    def update(df, market):
        for _, row in df.iterrows():
            sp = StockPrice(code=row['Code'], 
                            market=market, 
                            date=date_, 
                            open=row['Open'],
                            high=row['High'],
                            low=row['Low'],
                            close=row['Close'],
                            volume=row['Volume'])
            try:
                db_insert(curs, sp)
            except IntegrityError:
                pass          
        
    columns = {'티커':'Code','시가':'Open', '고가':'High', '저가':'Low', '종가':'Close', '거래량':'Volume'}
    df = krx.get_market_ohlcv(date_, market='KOSPI').reset_index()
    df = df.rename(columns=columns)
    if df.values.any():
        update(df, 'kospi')
        
    df = krx.get_market_ohlcv(date_, market='KOSDAQ').reset_index()
    df = df.rename(columns=columns)
    if df.values.any():
        update(df, 'kosdaq')
     
    for code in ['KS11', 'KQ11']:
        df = fdr.DataReader(code, date_, date_).reset_index()
        df['Code'] = code
        update(df, code)
        
    conn.commit()
    conn.close()    

In [108]:
# build_stock_price_db(DBFILE, '2022-01-01', '2024-07-31')

In [109]:
# update_stock_price_db(DBFILE)

In [112]:
oneday_update_stock_price_db(DBFILE)

1 001 2024-08-03 00:00:00 2026-08-02 00:00:00
no data or code("1001") not found
2 001 2024-08-03 00:00:00 2026-08-02 00:00:00
no data or code("2001") not found


In [235]:
def strong_stocks(curs: Cursor, market: str):
    if market.lower() == 'kospi':
        market_ = 'KS11'
    elif market.lower() == 'kosdaq':
        market_ = 'KQ11'
    else:
        assert False, "wrong market"
    
    df = get_ohlc(curs, market_)
    df['Date'] = pd.to_datetime(df['Date'])
    df_market_index = df.set_index('Date')    
    df_market_index_week = resample_df(df_market_index, period='W-FRI')[::-1]
    date = df_market_index_week[:52]["Low"].idxmin()
    lowest = df_market_index_week.loc[date, "Low"]
    currval = df_market_index_week.iloc[0]["Close"] 
    ks_rate = (currval - lowest)/lowest*100
    res = []
    codes = [x[0] for x in get_stock_list(curs, market.lower())]
    last_date = df_market_index.index.max()
    start_date = last_date - timedelta(days=400)
    df_marcap = krx.get_market_cap(last_date.strftime('%Y-%m-%d'))
    for code in tqdm(codes):
        df = get_ohlc(curs, code, start_date.strftime('%Y-%m-%d'), last_date.strftime('%Y-%m-%d'))
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.set_index('Date')
        df_week = resample_df(df, period='W-FRI')[::-1]
        if len(df_week) < 52:
            continue
        date = df_week[:52]["Low"].idxmin()        
        lowest = df_week.loc[date, "Low"]
        if lowest == 0:
            continue
        currval = df["Close"].iloc[-1]
        rate = (currval - lowest)/lowest*100
        if (rate > ks_rate):            
            res.append([code, rate, df_marcap.loc[code, "시가총액"]])
    
    return ks_rate, pd.DataFrame(res, columns=["code", "rate", "marcap"])

In [236]:
conn = sqlite3.connect(DBFILE)
curs = conn.cursor()

In [242]:
ks_rate, df = strong_stocks(curs, 'kosdaq')

100%|██████████████████████████████████████████████████████████████████████████████| 1746/1746 [00:23<00:00, 74.96it/s]


In [243]:
df[df['rate'] > 30]

,code,rate,marcap
0,000250,223.423423,4210616224000
1,000440,48.355059,148828407000
5,001840,56.541020,69914332800
8,002800,48.817204,104979860000
11,005160,41.676505,325466892000
...,...,...,...
839,900290,36.842105,201451250000
842,900340,146.099291,83233897580
844,950140,86.263736,336760328700
845,950160,125.291829,1378513858050


In [241]:
get_ohlc(curs, '000100')

,Code,Market,Date,Open,High,Low,Close,Volume
0,000100,kospi,2022-01-03,56854.0,58135.0,56762.0,58045.0,174870.0
1,000100,kospi,2022-01-04,58319.0,58319.0,56762.0,56946.0,175334.0
2,000100,kospi,2022-01-05,56578.0,56670.0,55479.0,55938.0,220258.0
3,000100,kospi,2022-01-06,55113.0,56304.0,55023.0,55023.0,183085.0
4,000100,kospi,2022-01-07,55021.0,56211.0,54655.0,55663.0,162565.0
...,...,...,...,...,...,...,...,...
633,000100,kospi,2024-07-30,96300.0,97600.0,94600.0,97600.0,794408.0
634,000100,kospi,2024-07-31,98100.0,99000.0,89500.0,94400.0,1779758.0
635,000100,kospi,2024-08-01,93300.0,95000.0,91700.0,92000.0,721342.0
636,000100,kospi,2024-08-02,90600.0,92100.0,89900.0,90800.0,606631.0


In [ ]:
conn.close()